In [8]:
%matplotlib inline
import os
from os.path import join as oj
import sys, time
import pickle as pkl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('precision', 3)

In [10]:
save_path = "../results_for_export"
# trained_model_folder = '../models/ColorMNIST'
trained_model_folder = '../models/DecoyMNIST'
fnames = [oj(trained_model_folder, fname) for fname in os.listdir(trained_model_folder)]
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results = pd.concat(results_list, axis=1).T.infer_objects() 

In [11]:
results['final_acc'] = [x[-1] for x in results['accs_test']] 
results['final_acc_train'] = [x[-1] for x in results['accs_train']]
results['final_cd'] = [x[-1] for x in results['cd']]
results['final_test_loss'] = [min(x) for x in results['losses_test']]
results['final_train_loss'] = [min(x) for x in results['losses_train']]


In [12]:
results.reset_index(drop=True, inplace=True)
results_save = results[['regularizer_rate','final_acc','final_acc_train','final_test_loss', 'final_cd', 'method', 'dataset', ]].sort_values(by = ['regularizer_rate'])

In [18]:
concise_results = results_save.groupby(by = ['dataset', 'method','regularizer_rate']).std()
concise_results.columns = ['Test accuracy', 'Train accuracy', 'Test loss' , 'CD',]
concise_results

Test accuracy  Train accuracy  \
dataset method       regularizer_rate                                  
Decoy   CDEP         1.0                       3.862           0.856   
                     10.0                      5.053           0.807   
                     100.0                     0.712           0.807   
                     1000.0                    0.754           1.276   
                     10000.0                   0.137           0.988   
        ExpectedGrad 0.0                       5.145           1.276   
                     0.1                       0.170           1.804   
                     1.0                       0.304           2.992   
                     10.0                     14.899           1.562   
                     100.0                     0.000           0.000   
                     1000.0                    0.000           0.000   
                     100000.0                  0.000           0.000   
        Grad         1.0                       2.363           0.807   
                     10.0                      2.454           1.276   
                     100.0                     3.911           0.856   
                     1000.0                    0.093           0.856   
                     10000.0                   0.103           0.807   

                                       Test loss         CD  
dataset method       regularizer_rate                        
Decoy   CDEP         1.0               1.190e-03  1.782e-04  
                     10.0              4.117e-03  7.621e-06  
                     100.0             9.994e-04  2.814e-06  
                     1000.0            7.909e-03  1.618e-07  
                     10000.0           4.487e-03  4.692e-10  
        ExpectedGrad 0.0               2.491e-03  0.000e+00  
                     0.1               6.165e-03  1.359e-01  
                     1.0               1.128e-02  3.095e-02  
                     10.0              2.733e-01  1.827e-02  
                     100.0             7.072e-02  2.443e-03  
                     1000.0            1.264e+00  2.473e-05  
                     100000.0          2.394e-01  1.679e-06  
        Grad         1.0               1.140e-03  6.232e-04  
                     10.0              3.391e-03  4.959e-04  
                     100.0             3.015e-03  2.580e-05  
                     1000.0            2.778e-03  3.234e-07  
                     10000.0           5.764e-03  2.775e-07

In [19]:
results_save.groupby(by = ['dataset', 'method','regularizer_rate']).mean()

final_acc  final_acc_train  \
dataset method       regularizer_rate                               
Decoy   CDEP         1.0                  72.952           99.219   
                     10.0                 76.933           99.479   
                     100.0                95.712           99.479   
                     1000.0               97.235           96.354   
                     10000.0              96.688           95.312   
        ExpectedGrad 0.0                  60.970           98.438   
                     0.1                  97.767           95.833   
                     1.0                  93.830           92.969   
                     10.0                 43.160           15.625   
                     100.0                10.090            4.688   
                     1000.0               10.090            4.688   
                     100000.0             10.090            4.688   
        Grad         1.0                  56.930           99.479   
                     10.0                 63.997           98.958   
                     100.0                92.418           99.219   
                     1000.0               98.893           99.219   
                     10000.0              98.942           98.958   

                                       final_test_loss   final_cd  
dataset method       regularizer_rate                              
Decoy   CDEP         1.0                         0.016  9.805e-05  
                     10.0                        0.019  8.109e-06  
                     100.0                       0.026  2.279e-06  
                     1000.0                      0.056  4.606e-07  
                     10000.0                     0.112  4.433e-10  
        ExpectedGrad 0.0                         0.015  0.000e+00  
                     0.1                         0.077  4.478e-01  
                     1.0                         0.199  1.364e-01  
                     10.0                        1.708  3.754e-02  
                     100.0                       6.615  2.694e-03  
                     1000.0                     11.753  5.038e-04  
                     100000.0                   16.699  7.962e-06  
        Grad         1.0                         0.015  2.704e-04  
                     10.0                        0.017  2.941e-04  
                     100.0                       0.025  1.610e-05  
                     1000.0                      0.031  4.680e-07  
                     10000.0                     0.031  3.384e-07

In [ ]:
plot = sns.lineplot(x ='regularizer_rate' , y='final_acc', data = results_save[results_save.method == 'CDEP'],label="CDEP",legend = "brief"  );
plot = sns.lineplot(x ='regularizer_rate' , y='final_acc', data = results_save[results_save.method == 'Grad'], label="RRR" );
plot = sns.lineplot(x ='regularizer_rate' , y='final_acc', data = results_save[results_save.method == 'ExpectedGrad'], label="Expected Gradients");
plot = sns.lineplot(x ='regularizer_rate' , y='final_acc', data = results_save[results_save.method == 'EGradients'], label="Expected Gradients");
plot.set(xscale="log")
plot.set_xlabel("Regularization strength")
plot.set_ylabel("Test Accuracy")
plot.set_title("");
fig  = plot.get_figure()
fig.tight_layout()
# fig.savefig(oj(save_path,"ColorMNIST_results"))
plt.show()